In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import cv2
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.linear_model import Lasso
# from scikeras.wrappers import KerasClassifier, KerasRegressor
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

# Read datasets

In [2]:
readDataPath = './resources/'
# Read datasets
landmarksJaffe = pd.read_csv(readDataPath+'features/landmarksJaffe.csv')
print(f"landmarksJaffe shape: {landmarksJaffe.shape}")
landmarksKdef = pd.read_csv(readDataPath+'features/landmarksKDEF.csv')
print(f"landmarksKdef.shape: {landmarksKdef.shape}")
landmarksCKplus = pd.read_csv(readDataPath+'features/landmarksCKPLUS.csv')
print(f"landmarksCKplus.shape: {landmarksCKplus.shape}")
landmarksAffectnet = pd.read_csv(readDataPath+'features/landmarksAFFECTNET.csv')
print(f"landmarksAffectnet.shape: {landmarksAffectnet.shape}")
landmarksFER2013 = pd.read_csv(readDataPath+'features/landmarksFER2013.csv')
print(f"landmarksFER2013.shape: {landmarksFER2013.shape}")

landmarksJaffe shape: (213, 207)
landmarksKdef.shape: (2917, 207)
landmarksCKplus.shape: (840, 207)
landmarksAffectnet.shape: (24820, 207)
landmarksFER2013.shape: (27478, 207)


# Concatenate 3D landmark datasets

In [3]:
# stack datasets
emotionLandmarks = pd.concat([landmarksJaffe, landmarksKdef, landmarksCKplus, landmarksAffectnet, landmarksFER2013], axis=0)
print(f"emotionLandmarks shape: {emotionLandmarks.shape}")

emotionLandmarks shape: (56268, 207)


# Split 3D landmark Training data & Labels

In [4]:
X_landmarks = emotionLandmarks.drop('emotion', axis=1)
y = emotionLandmarks['emotion'].astype('int8')
print(f"X_landmarks shape: {X_landmarks.shape}")

X_landmarks shape: (56268, 206)


# Scaling 3D landmarks

In [5]:
scaler = MinMaxScaler()

# 3D landmark scaling
X_landmarks_scaled = scaler.fit_transform(X_landmarks)
print(f"X_3D_scaled Shape = {np.shape(X_landmarks_scaled)}")
# print(f"scaler min = {scaler.data_min_}, scaler max = {scaler.data_max_}")

# # save min/max values of each feature
# with open('./datasets/combiModels/scalerParams_svm_mvp1_fer2013.pkl', 'wb') as file:
#     params = {'minValues': scaler.data_min_, 'maxValues': scaler.data_max_}
#     pickle.dump(params, file)
# file.close()

X_3D_scaled Shape = (56268, 206)


# Dimensionality Reduction of 3D landmark dataset

In [6]:
n_components = 120
pca = PCA(n_components= n_components)
X_landmarks_scaled_pca = pca.fit_transform(X_landmarks_scaled)
# print(f"X_3D_scaled_pca shape: {X_landmarks_scaled_pca.shape}, type: {type(X_landmarks_scaled_pca)}")

# # Save the fitted PCA model to a file
# with open(readDataPath+'combiModels/pca_svm_mvp1_fer2013.pkl', 'wb') as file:
#     pickle.dump(pca, file)
# file.close()

print(f"Explained variance: {round(np.cumsum(pca.explained_variance_ratio_ * 100)[-1],2)}%")

Explained variance: 98.74%


# Shuffle datasets

In [7]:
print(f"X_landmarks_scaled_pca shape: {X_landmarks_scaled_pca.shape}, type: {type(X_landmarks_scaled_pca)}")
print(f"y shape: {y.shape}, type: {type(y)}")

# shuffle
X_landmarks_scaled_pca_shuffled, y_shuffled = shuffle(X_landmarks_scaled_pca, y, random_state=512)

print(f"X_landmarks_scaled_pca_shuffled shape: {X_landmarks_scaled_pca_shuffled.shape}, type: {type(X_landmarks_scaled_pca_shuffled)}")
print(f"y shape: {y_shuffled.shape}, type: {type(y_shuffled)}")

X_landmarks_scaled_pca shape: (56268, 120), type: <class 'numpy.ndarray'>
y shape: (56268,), type: <class 'pandas.core.series.Series'>
X_landmarks_scaled_pca_shuffled shape: (56268, 120), type: <class 'numpy.ndarray'>
y shape: (56268,), type: <class 'pandas.core.series.Series'>


# Check label count

In [8]:
# string labels
emo = ['angry', 
        'disgust',
        'fear',
        'happy',
        'sad',
        'surprised',
        'neutral'
]

# count classes
emotions, counts =np.unique(y_shuffled, return_counts=True)
for c, count in zip(emotions, counts):
    print(f"{emo[c]} count = {count}")

angry count = 7752
disgust count = 4425
fear count = 7307
happy count = 12546
sad count = 7645
surprised count = 8358
neutral count = 8235


# Build and Validate Single run SVM-RBF

In [9]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_landmarks_scaled_pca_shuffled, y_shuffled, test_size=0.1, shuffle=True, random_state=190)

# train model
svmClf = SVC(kernel ='rbf', C=7.5, gamma=0.35)
hist = svmClf.fit(X_train, y_train)

# Calculate training accuracy
y_train_pred = svmClf.predict(X_train)
train_acc = accuracy_score(y_train, y_train_pred)

# Calculate validation accuracy
y_val_pred = svmClf.predict(X_test)
val_acc = accuracy_score(y_test, y_val_pred)

print(f"Training accuracy: {round(train_acc*100,2)}%, Validation accuracy: {round(val_acc*100,2)}%")

Training accuracy: 77.76%, Validation accuracy: 58.13%


# Confusion Matrix Typical Case

In [10]:
y_pred = svmClf.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
TP = cm.diagonal().sum()
totalIns = cm.sum()

print(f"Number of Test Instances= {totalIns}")
print(f"Accuracy= {round((TP/totalIns)*100,2)}%")
print('Confusion Matrix')
print(cm)

Number of Test Instances= 5627
Accuracy= 58.13%
Confusion Matrix
[[ 433   56   41   33   90   22  106]
 [  83  203   15   53   36   25   37]
 [ 109   22  259   37   95  142   68]
 [  28   27   19 1072   24   40   41]
 [  89   38   60   56  362   26  162]
 [  34   17   92   48   25  520   68]
 [ 104   31   44   44  117   52  422]]


# Grid Search Best Estimator for SVM-RBF

In [11]:
pipeline = Pipeline([
                        ('svm', SVC(kernel ='rbf', probability=True, random_state=42)),
])

# define the hyperparameters to search over
param_grid = {
    'svm__C': [7.5],
    'svm__gamma': [0.35],
}

# perform a grid search to find the best hyperparameters
grid_search = GridSearchCV(pipeline, param_grid=param_grid, cv=10, verbose=2)
hist = grid_search.fit(X_landmarks_scaled_pca_shuffled, y_shuffled)

# print the best hyperparameters and the corresponding score
print(f"Best hyperparameters: {grid_search.best_params_}")
print(f"Best score: {round(grid_search.best_score_,4)*100}%")

Fitting 10 folds for each of 1 candidates, totalling 10 fits
[CV] END ........................svm__C=7.5, svm__gamma=0.35; total time=16.6min
[CV] END ........................svm__C=7.5, svm__gamma=0.35; total time=16.9min
[CV] END ........................svm__C=7.5, svm__gamma=0.35; total time=17.0min
[CV] END ........................svm__C=7.5, svm__gamma=0.35; total time=16.7min
[CV] END ........................svm__C=7.5, svm__gamma=0.35; total time=68.2min
[CV] END ........................svm__C=7.5, svm__gamma=0.35; total time=17.2min
[CV] END ........................svm__C=7.5, svm__gamma=0.35; total time=17.7min
[CV] END ........................svm__C=7.5, svm__gamma=0.35; total time=17.3min
[CV] END ........................svm__C=7.5, svm__gamma=0.35; total time=17.8min
[CV] END ........................svm__C=7.5, svm__gamma=0.35; total time=17.7min
Best hyperparameters: {'svm__C': 7.5, 'svm__gamma': 0.35}
Best score: 58.06%


# K-Fold Cross Validation

In [12]:
k=10
svmClfCV = SVC(kernel ='rbf', C=7.5, gamma=0.35)
scores = cross_val_score(svmClfCV, X_landmarks_scaled_pca_shuffled, y_shuffled, cv=k)
print(f"Mean Cross-Validation Score: {round(np.mean(scores)*100,2)}%") #84.59

Mean Cross-Validation Score: 58.06%


# Save Serialized Model

In [13]:
# bestModel = grid_search.best_estimator_
# with open(readDataPath+'combiModels/modelSvmRbf_mvp1_fer2013.pkl', 'wb') as file:
#     pickle.dump(bestModel,file)

# file.close()